### Importing libraries and setup Azure OpenAI

In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings
from langchain.tools import Tool

from IPython.display import HTML, display
import pandas as pd
import markdown2
import datetime

import ipywidgets as widgets
from typing import Union, Callable

In [2]:
# Update the client initialization cell
load_dotenv()

# Initialize the Azure OpenAI model with LangChain
llm = AzureChatOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_DEPLOYMENT_NAME"),
    api_version="2025-03-01-preview"
)

### Initialize  RCA Technique advisor agent and establish knowledge base for it

In [3]:
# Load and process the PDF
loader = PyPDFLoader("data\IEC-62740.pdf")  # Adjust filename as needed
documents = loader.load()

# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
chunks = text_splitter.split_documents(documents)

# Create embeddings and vector store
embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=os.getenv("EMBEDDING_ENDPOINT"),
    api_key=os.getenv("EMBEDDING_API_KEY"),
    deployment=os.getenv("EMBEDDING_DEPLOYMENT"),
    api_version="2025-03-01-preview"
)

vectorstore = FAISS.from_documents(chunks, embeddings) # Vectorstore creation - FAISS from Meta

# Create a retrieval tool
def retrieve_rca_techniques(query: str) -> str:
    """Search for relevant RCA techniques in the knowledge base."""
    docs = vectorstore.similarity_search(query, k=3)
    return "\n\n".join(doc.page_content for doc in docs)

rca_knowledge_tool = Tool(
    name="rca_techniques_lookup",
    description="Search for industry standard RCA techniques and their applications",
    func=retrieve_rca_techniques
)

<>:2: SyntaxWarning: invalid escape sequence '\I'
<>:2: SyntaxWarning: invalid escape sequence '\I'
C:\Users\ABHANA\AppData\Local\Temp\ipykernel_6188\3999924126.py:2: SyntaxWarning: invalid escape sequence '\I'
  loader = PyPDFLoader("data\IEC-62740.pdf")  # Adjust filename as needed


In [4]:
# Define the RCA Technique Advisor agent
technique_advisor_prompt = """You are an expert in Root Cause Analysis (RCA) techniques.
Given an incident description:
1. Use the rca_techniques_lookup tool to search for relevant techniques
2. Consider the incident context and complexity
3. Suggest the most appropriate RCA technique
4. Justify your choice with reference to industry standards
5. Format your response in Markdown with clear headings

Current incident: {input}
"""

technique_advisor = create_react_agent(
    model=llm,
    tools=[rca_knowledge_tool],
    name="rca_technique_advisor",
    prompt=technique_advisor_prompt
)

### Initialize RCA Performer agent and provide it with access to investigation reports/questionnaire

In [5]:
def process_investigation_report(file_path: str) -> str:
    """Process investigation report and extract relevant information."""
    try:
        loader = PyPDFLoader(file_path)
        pages = loader.load()
        return "\n\n".join(page.page_content for page in pages)
    except Exception as e:
        return f"Error processing report: {str(e)}"

investigation_tool = Tool(
    name="investigation_report_lookup",
    description="Search through investigation reports for relevant evidence and context",
    func=process_investigation_report
)

In [6]:
# Define the RCA Performer agent
rca_performer_prompt = """You are an RCA specialist. Given an incident description and a recommended RCA technique:
1. If interview evidence is available, use it to inform your analysis
2. If investigation reports are available, use them as evidence
3. Analyze the incident using the recommended technique and available evidence
4. Present your analysis in a structured format with clear sections:
   - Incident Overview
   - Evidence Used
   - Analysis Process
   - Root Causes Identified
   - Supporting Evidence
   - Recommendations

Format your response in Markdown for better readability.
"""

rca_performer = create_react_agent(
    model=llm,
    tools=[investigation_tool],
    name="rca_performer",
    prompt=rca_performer_prompt
)


In [7]:
def interactive_interview(incident_description: str, on_complete: Callable[[dict], None]):
    """Conduct an interactive interview and call on_complete when done."""
    questions = {
        "maintenance": [
            "What maintenance procedures were being followed?",
            "Who was involved in the maintenance activity?",
            "What safety measures were in place?",
            "Were there any similar incidents in the past?"
        ],
        "equipment_failure": [
            "What was the equipment's maintenance history?",
            "When was the last inspection?",
            "Were there any warning signs before the failure?",
            "What were the operating conditions at the time?"
        ]
    }

    question_set = "maintenance" if "maintenance" in incident_description.lower() else "equipment_failure"
    questions_list = questions[question_set]

    state = {
        "current_q_idx": 0,
        "responses": {}
    }

    question_label = widgets.HTML(f"<h3>Q: {questions_list[0]}</h3>")
    answer_box = widgets.Textarea(placeholder='Type your answer here...', layout=widgets.Layout(width='100%', height='100px'))
    next_button = widgets.Button(description='Next Question', button_style='primary')
    progress = widgets.HTML(value=f"Question 1 of {len(questions_list)}")
    status = widgets.Output()

    def on_next_clicked(_):
        current_q = questions_list[state["current_q_idx"]]
        state["responses"][current_q] = answer_box.value

        state["current_q_idx"] += 1

        if state["current_q_idx"] < len(questions_list):
            question_label.value = f"<h3>Q: {questions_list[state['current_q_idx']]}</h3>"
            progress.value = f"Question {state['current_q_idx'] + 1} of {len(questions_list)}"
            answer_box.value = ""
            if state["current_q_idx"] == len(questions_list) - 1:
                next_button.description = "Finish"
        else:
            container.close()
            on_complete(state["responses"])

    next_button.on_click(on_next_clicked)

    container = widgets.VBox([
        widgets.HTML(f"<h2>RCA Interview</h2><p>Incident: {incident_description}</p>"),
        progress,
        question_label,
        answer_box,
        next_button,
        status
    ])
    display(container)


#### Function to format the output and export the result to a markdown file

In [8]:
def display_final_rca_response(result):
    # Create HTML template with Bootstrap and custom CSS
    html = """
    <div style="max-width: 800px; margin: 20px auto; background-color: white; padding: 20px; border-radius: 8px;">
        <style>
            .card-body { 
                padding: 15px;
                background-color: white;
            }
            .markdown-content { 
                font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, "Helvetica Neue", Arial, sans-serif;
                color: #333333;
            }
            .markdown-content h1, h2, h3 { 
                color: #2980b9; 
                margin-top: 20px;
                font-weight: 600;
            }
            .markdown-content pre { 
                background-color: #f8f9fa; 
                padding: 10px; 
                border-radius: 4px;
                border: 1px solid #e0e0e0;
            }
            .markdown-content table {
                width: 100%;
                border-collapse: collapse;
                margin: 15px 0;
            }
            .markdown-content th {
                background-color: #2980b9;
                color: white;
                padding: 10px;
                border: 1px solid #e0e0e0;
            }
            .markdown-content td {
                padding: 8px;
                border: 1px solid #e0e0e0;
                background-color: white;
            }
            .markdown-content ul, ol { 
                padding-left: 20px;
                color: #333333;
            }
        </style>
        <h2 style="color: #2980b9; text-align: center;">RCA Analysis Results</h2>
    """
    
    # Get the last message (supervisor's final response)
    final_message = result["messages"][-1]
    
    # Convert Markdown to HTML with tables support
    markdown_html = markdown2.markdown(
        final_message.content,
        extras=['tables', 'fenced-code-blocks']
    )
    
    # Save to markdown file
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"rca_analysis_{timestamp}.md"
    
    with open(filename, 'w', encoding='utf-8') as f:
        f.write("# RCA Analysis Results\n\n")
        f.write(final_message.content)
    
    print(f"\nAnalysis saved to: {filename}")
    
    # Add the final response card
    card_html = f"""
    <div class="card">
        <div class="card-body">
            <div class="markdown-content">
                {markdown_html}
            </div>
        </div>
    </div>
    """
    
    html += card_html + "</div>"
    
    # Display the HTML
    display(HTML(html))

### Initialize the supervisor agent

In [9]:
# Create the supervisor agent
supervisor_prompt = (
    "You are a supervisor managing two agents: 'rca_technique_advisor' and 'rca_performer'. "
    "First, delegate the incident description to 'rca_technique_advisor' to get the recommended RCA technique. "
    "Then, pass both the incident description and the recommended technique to 'rca_performer' to perform the analysis. "
    "Finally, compile and present the complete RCA findings."
)

supervisor = create_supervisor(
    agents=[technique_advisor, rca_performer],
    model=llm,
    prompt=supervisor_prompt
)

# Compile the supervisor into an executable application
app = supervisor.compile()

### Define the function to collect evidence from user/report

In [10]:
def analyze_incident(incident_description: str, report_path: Union[str, None] = None):
    """Analyze incident using either investigation reports or interactive interview."""
    from IPython.display import display
    import ipywidgets as widgets
    import os

    print("\n=== Starting RCA Analysis ===")

    loading = widgets.HTML("<b>Analyzing... please wait.</b> <i class='fa fa-spinner fa-spin'></i>")
    messages = [{"role": "user", "content": incident_description}]
    context = {}

    if report_path and os.path.exists(report_path):
        print("Using investigation report for analysis...")
        evidence = process_investigation_report(report_path)
        context["evidence_type"] = "report"
        context["evidence"] = evidence

        display(loading)
        result = app.invoke({
            "messages": messages,
            "context": context
        })
        loading.value = ""  # Stop spinner
        print("\n=== Analysis Results ===")
        display_final_rca_response(result)

    else:
        print("No investigation report found. Starting interactive interview...")

        def on_complete(responses):
            evidence = "\n\n".join([f"Q: {q}\nA: {a}" for q, a in responses.items()])
            context["evidence_type"] = "interview"
            context["evidence"] = evidence

            display(loading)
            result = app.invoke({
                "messages": messages,
                "context": context
            })
            loading.value = ""  # Stop spinner
            print("\n=== Analysis Results ===")
            display_final_rca_response(result)

        interactive_interview(incident_description, on_complete)


#### Analysis using the investigation report

In [11]:
# Test the interactive analysis
incident = "A dropped object (a grating plate) on the Heidrun TLP, on 22 September 2015."
report_path = "data\investigation_reports\investigation-report---statoil---heidrun.pdf"
result1 = analyze_incident(incident, report_path)  # This will use the investigation report

<>:3: SyntaxWarning: invalid escape sequence '\i'
<>:3: SyntaxWarning: invalid escape sequence '\i'
C:\Users\ABHANA\AppData\Local\Temp\ipykernel_6188\3369250320.py:3: SyntaxWarning: invalid escape sequence '\i'
  report_path = "data\investigation_reports\investigation-report---statoil---heidrun.pdf"



=== Starting RCA Analysis ===
Using investigation report for analysis...


HTML(value="<b>Analyzing... please wait.</b> <i class='fa fa-spinner fa-spin'></i>")


=== Analysis Results ===

Analysis saved to: rca_analysis_20250429_230557.md


#### Analysis based on questionnaire

In [15]:
result2 = analyze_incident(incident)  # This will trigger the interview


=== Starting RCA Analysis ===
No investigation report found. Starting interactive interview...
